In [46]:
import pandas as pd
from datetime import datetime

In [47]:
customer_df = pd.read_csv('./Datos/dimension_customer.csv', sep=',')
customer_df.set_index('Customer_Key', inplace=True)
customer_df

,Customer,Bill_To_Customer,Category,Buying_Group,Primary_Contact,Postal_Code
Customer_Key,,,,,,
1,Tailspin Toys (Head Office),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Waldemar Fisar,90410
2,Tailspin Toys (Sylvanite- MT),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Lorena Cindric,90216
3,Tailspin Toys (Peeples Valley- AZ),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Bhaargav Rambhatla,90205
4,Tailspin Toys (Medicine Lodge- KS),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Daniel Roman,90152
5,Tailspin Toys (Gasport- NY),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Johanna Huiting,90261
...,...,...,...,...,...,...
398,Wingtip Toys (Haycock- AK),Wingtip Toys (Head Office),Novelty Shop,Wingtip Toys,Bhaamini Kanaparthi,90603
399,Wingtip Toys (Kapa'a- HI),Wingtip Toys (Head Office),Novelty Shop,Wingtip Toys,Margit Molnar,90796
400,Wingtip Toys (Dickworsham- TX),Wingtip Toys (Head Office),Novelty Shop,Wingtip Toys,Chandni Jandhyala,90704


In [48]:
new_customer_df = pd.read_csv('./Datos/dimension_customer_new.csv', sep=',')
new_customer_df.set_index('Customer_Key', inplace=True)
new_customer_df

,Customer,Bill_To_Customer,Category,Buying_Group,Primary_Contact,Postal_Code
Customer_Key,,,,,,
0,Unknown,NaN,NaN,NaN,NaN,NaN
1,Tailspin Toys (Head Office),Tailspin Toys (Head Office),Toys Shop,Tailspin Toys,Waldemar Fisar,90410.0
2,Tailspin Toys (Sylvanite- MT),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Lorena Cindric,90216.0
3,Tailspin Toys (Peeples Valley- AZ),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Bhaargav Rambhatla,90205.0
4,Tailspin Toys (Medicine Lodge- KS),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Daniel Roman,90152.0
...,...,...,...,...,...,...
398,Wingtip Toys (Haycock- AK),Wingtip Toys (Head Office),Novelty Shop,Wingtip Toys,Bhaamini Kanaparthi,90603.0
399,Wingtip Toys (Kapa'a- HI),Wingtip Toys (Head Office),Novelty Shop,Wingtip Toys,Margit Molnar,90796.0
400,Wingtip Toys (Dickworsham- TX),Wingtip Toys (Head Office),Toys Shop,Wingtip Toys,Chandni Jandhyala,90704.0


# Métodos SCD

Utilizamos la siguiente referencia para guiarnos: 

## Tipo 0

Acorde a lo visto en el material de clase, se puede aplicar este método. En particular, no toca hacer nada.

## Tipo 1

In [49]:
customer_df_type1 = customer_df.copy()
new_customer_df_type1 = new_customer_df.copy()

In [50]:
for index in new_customer_df_type1.index:
  if index in customer_df_type1.index:
    customer_df_type1.loc[index] = new_customer_df_type1.loc[index]

  else:
    customer_df_type1.loc[index] = new_customer_df_type1.loc[index]

customer_df_type1.to_csv('./Datos/dimension_customer_type1.csv', index=False)

## Tipo 2

In [54]:
customer_df_type2 = customer_df.copy()
new_customer_df_type2 = new_customer_df.copy()

In [55]:
customer_df_type2['_version'] = 1
new_customer_df_type2['_version'] = 1

In [56]:
for index in new_customer_df_type2.index:
  if index in customer_df_type2.index:
    previous = customer_df_type2.loc[index]
    current = new_customer_df_type2.loc[index]

    if previous.equals(current) is False:
      new_customer_df_type2.loc[index, '_version'] = customer_df_type2.loc[index, '_version'] + 1

    else:
      new_customer_df_type2.drop(index)

customer_df_type2 = customer_df_type2.append(new_customer_df_type2)
customer_df_type2.to_csv('./Datos/dimension_customer_type2.csv', index=False)

## Tipo 3

Asumimos que el atributo que cambia es el contacto primario, por lo que es el más volátil de los presentados en la tabla.

In [57]:
customer_df_type3 = customer_df.copy()
new_customer_df_type3 = new_customer_df.copy()

In [58]:
customer_df_type3['Previous_Primary_Contact'] = 'Non-existent'

In [59]:
for index in new_customer_df_type3.index:
  if index in customer_df_type3.index:
    previous = customer_df_type3.loc[index, 'Primary_Contact']
    current = new_customer_df_type3.loc[index, 'Primary_Contact']

    if previous != current:
      customer_df_type3.loc[index, 'Primary_Contact'] = current
      customer_df_type3.loc[index, 'Previous_Primary_Contact'] = previous

  else:
    row = new_customer_df_type3.loc[index]
    row['Previous_Primary_Contact'] = 'Non-existent'
    customer_df_type3.loc[index] = row
  
customer_df_type3.to_csv('dimension_customer_type3.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
